In [1]:
# !stubber switch stable

In [2]:
! cd d:\mypython\micropython-stubber
!stubber docstubs 


22:31:50 | ℹ️ micropython-stubber 1.24.0
22:31:51 | ℹ️ Get docstubs for MicroPython v1.24.1
22:31:51 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_1-docstubs\array\__init__.pyi
22:31:51 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_1-docstubs\asyncio\__init__.pyi
22:31:51 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_1-docstubs\binascii\__init__.py
i
22:31:51 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_1-docstubs\bluetooth\__init__.p
yi
22:31:51 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_1-docstubs\btree\__init__.pyi
22:31:51 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_1-docstubs\cmath\__init__.pyi
22:31:51 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_1-docstubs\collections\__init__
.pyi
22:31:51 | ℹ️  - Writing to: 
repos\micropython-stubs\stubs\micropython-v1_24_1-docstubs\cryptolib\__init__.p
yi
22:31:51 | ℹ️  - Writ

In [3]:
import shutil
from pathlib import Path

docstubs = Path("repos/micropython-stubs/stubs/micropython-v1_24_1-docstubs")
reference = Path("repos/micropython-stubs/micropython-reference")

In [ ]:
# Enrich docstubs

!stubber -VV enrich --params-only --source {reference} --dest {docstubs} 

In [ ]:
def cp_stubmod(module: str, target: Path | None = None) -> None:
    target = target or docstubs
    src = reference / module
    dst = target / module
    return shutil.copytree(src, dst, dirs_exist_ok=True)


def cp_stubfile(module: str, target: Path | None = None) -> None:
    target = target or docstubs
    if target.is_dir():
        target.mkdir(parents=True, exist_ok=True)
    src = reference / module
    dst = target / module
    return shutil.copy2(src, dst)


# TODO: Fix rp2 / _rp2
# print(cp_stubmod("rp2"))
print(cp_stubfile("rp2/PIOASMEmit.pyi", docstubs))
print(cp_stubmod("_mpy_shed"))
print(cp_stubmod("asyncio"))

In [95]:
# run autoflake on all files - one by one to get the __init__.pyi files
for f in docstubs.rglob("*.pyi"):
    ! autoflake {f} --in --imports typing_extensions,_mpy_shed,_typeshed


In [ ]:

! isort {docstubs}

In [ ]:
!pyright {docstubs}

- 13 errors, 18 warnings, 2 informations 
- 9 errors, 18 warnings, 2 informations 
- 11 errors, 18 warnings, 2 informations 
- 8 errors, 18 warnings, 2 informations 
- 8 errors, 17 warnings, 2 informations 
- 18 errors, 15 warnings, 2 informations :-( 
- 12 errors, 14 warnings, 2 informations 
- 8 errors, 14 warnings, 2 informations 
- 0 errors, 17 warnings, 2 informations 
- 0 errors, 10 warnings, 2 informations 
- 0 errors, 0 warnings, 2 informations  Needs  manual copy of FOO = const(0)
- 0 errors, 12 warnings, 2 informations ( better merging of overloads --> more unresolved references)

In [98]:
import shutil
shutil.rmtree(docstubs / "_mpy_shed" , ignore_errors=True)


In [99]:
# !stubber frozen --version stable

In [ ]:
# Werkt goed zonder 'enrich-ed' docstubs
!stubber merge --port rp2 --version stable

In [ ]:
merged_p = Path("repos/micropython-stubs/stubs/micropython-v1_24_1-rp2-RPI_PICO-merged")
# add mpshed to test the quality of the merge
print(cp_stubmod("_mpy_shed", merged_p))

# remove the empty non functional collections.pyi
print(( merged_p / "collections.pyi").unlink())


In [ ]:
from stubber.rst.lookup import  U_MODULES

for mod in U_MODULES:
    try:
        shutil.copy2(merged_p / f"{mod}.pyi", merged_p / f"u{mod}.pyi")
    except FileNotFoundError:
        print(f"replacing umodule with module: {mod}.pyi not found")



In [ ]:
!pyright {merged_p}

- 86 errors, 35 warnings, 0 informations 
rp2
- 88 errors, 33 warnings, 0 informations 
- 1 error, 4 warnings, 0 informations 
- 3 errors, 8 warnings, 0 informations 
- 2 errors, 8 warnings, 0 informations 

In [103]:
# !stubber build --port esp32